In [3]:
#Scrape and download all links to the local file system due to http speed limit
import urllib.request
html_doc = ''
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_Greek_mythological_figures') as response:
    html_doc = response.read()

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')
all_links = soup.find_all('a')
print(len(all_links))

2090


In [5]:
#Download all of the relevant web pages from all the links
def download_pages():
    counter = 0
    for link in all_links:
        if(link.has_attr('href') and "/wiki/" in link['href']):
            if counter/len(all_links) % 10 == 0:
                print(str(counter/len(all_links) * 100) + '%')
            href = link['href']
            if '.jpg' or '.svg' or '.png' in href:
                continue
            urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/')+1:] + '.html')
    href = "/wiki/List_of_Greek_mythological_figures"
    urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/')+1:] + '.html')

In [6]:
soup.h3.span.text
soup.h3.next_sibling
sibling = soup.h3
counter = 0
h3_tag = soup.h3
h3_sibling_content = soup.h3.next_sibling.next_sibling

#Go back 1 sibling to be able to grab the first h3 tag into the list
clean_siblings = [s for s in h3_tag.previous_sibling.next_siblings if (str(s).strip() != "" and (s.name == "h3" or s.name == "div" or s.name == "table" or s.name=="ul"))]
clean_siblings = clean_siblings[:-7]
clean_siblings.remove(clean_siblings[5])

it = iter(clean_siblings)
heading_content_dict= {}
for sibling in it:
    heading_content_dict[sibling.text.replace("[edit]","").replace("/"," & ")] = next(it)
print(len(heading_content_dict))
#print(heading_content_dict)
print(heading_content_dict.keys())
        
    #print(next(it))

19
dict_keys(['Major gods and goddesses', 'Primordial deities', 'Titans and Titanesses', 'Gigantes and other "giants"', 'Personified concepts', 'Chthonic deities', 'Sea deities', 'Sky deities', 'Rustic deities', 'Agricultural deities', 'Health deities', 'Other deities', 'Deified mortals', 'Heroes', 'Notable women', 'Kings', 'Seers & oracles', 'Amazons', 'Inmates of Tartarus'])


In [7]:
import re

doc = heading_content_dict["Major gods and goddesses"]
soup_major = BeautifulSoup(str(doc), 'html.parser')
values = soup_major.find_all("td")
sub_pages = []

prime_list = []
for td in values:
    if not td.a.img:
        print(td.a['href'])
        f = open(td.a['href'][1:] + ".html", 'r')
        sub_doc = f.read()
        soup_major_sub = BeautifulSoup(str(sub_doc), 'html.parser')
        box_info = soup_major_sub.find_all("tr")
        for tr in box_info:
            if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents"):
                print(tr.th.text + ":")
                st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
                #st = tr.td.text
                result = [x.strip() for x in re.sub(r'(\[|\().*?(\]|\))','',st).split(",") if x != '']
                print(result) #Parse out citation square brackets
        print()

/wiki/Aphrodite
Children:
['Eros', 'Phobos', 'Deimos', 'Harmonia', 'Pothos', 'Anteros', 'Himeros', 'Hermaphroditus', 'Rhodos', 'Eryx', 'Peitho', 'Eunomia', 'The Graces', 'Priapus', 'Aeneas']
Parents:
['Uranus', 'Zeus', 'Dione']

/wiki/Apollo
Children:
['Asclepius', 'Troilus', 'Aristaeus', 'Orpheus']
Parents:
['Zeus', 'Leto']

/wiki/Ares
Children:
['Erotes', 'Phobos', 'Deimos', 'Phlegyas', 'Harmonia', 'Enyalios', 'Thrax', 'Oenomaus', 'Amazons', 'Adrestia']
Parents:
['Zeus', 'Hera']

/wiki/Artemis
Parents:
['Zeus', 'Leto']

/wiki/Athena
Parents:
['Metis', 'Zeus']

/wiki/Demeter
Children:
['Persephone', 'Despoina', 'Arion', 'Plutus', 'Philomelus', 'Eubuleus', 'Chrysothemis', 'Amphitheus I']
Parents:
['Cronus', 'Rhea']

/wiki/Dionysus
Children:
['Priapus', 'Hymen', 'Thaos', 'Staphylus', 'Oenopion', 'Comus', 'Phthonus', 'the Graces']
Parents:
['Zeus', 'Persephone', 'Zeus', 'Semele']

/wiki/Hades
Children:
['Macaria', 'Melinoe']
Parents:
['Cronus', 'Rhea']

/wiki/Hephaestus
Children:
['Thali

In [33]:
import re

doc = heading_content_dict["Primordial deities"]
soup_prime = BeautifulSoup(str(doc), 'html.parser')
values = soup_prime.find_all("tr")
links = []

prime_set = set()

prime_parents = {}
prime_children = {}

prime_uris = []
for tr in values:
    if tr.contents[3].a:
        f = open(tr.contents[3].a['href'][1:] + '.html','r')
        prime_uris.append(tr.contents[3].a['href'][1:])
        sub_doc = f.read()
        soup_prime_sub = BeautifulSoup(str(sub_doc), 'html.parser')
        box_info = soup_prime_sub.find_all("tr")
        print(tr.contents[3].a['href'])
        vr = re.sub(r'(\[|\().*?(\]|\))','',tr.contents[3].a['href']).strip().replace('_','')
        prime_set.add(vr[vr.rfind("/") + 1:])
        for tr in box_info:
            if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents" or tr.th.text == "Offspring"):
                st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
                #st = tr.td.text
                result = [x for x in re.split(r',|\n', re.sub(r'(\[|\().*?(\]|\))','',st).strip()) if x != ''] 
                v = re.sub(r'(\[|\().*?(\]|\))','',tr.contents[3].a['href']).strip().replace('_','')
                print(tr.th.text + ":")
                print(result) #Parse out citation square brackets
                
                if tr.th.text == "Children" or tr.th.text == "Offspring":
                    prime_children[vr[vr.rfind("/") + 1:]] = result
                elif tr.th.text == "Parents":
                    prime_parents[vr[vr.rfind("/") + 1:]] = result
        print()

print(prime_parents)
print("---")
print(prime_children)
print("----")
print(prime_set)

missing_parents = []
missing_children = []
for p in prime_set:
    if p not in prime_parents:
        #print("No Parent(s): " + p)
        missing_parents.append(p)
    if p not in prime_children:
        #print("No Children(s):" + p)
        missing_children.append(p)


        

/wiki/Achlys
Parents:
['Nyx']

/wiki/Aion_(deity)

/wiki/Aether_(mythology)
Children:
['Gaia', ' Thalassa', ' Uranus', ' Aergia', ' Pontus', ' Tartarus']
Parents:
['Erebus', 'Nyx ', 'Chronos', 'Ananke ', 'Chaos']

/wiki/Ananke_(mythology)
Offspring:
['Chaos', ' Aether', ' Phanes', ' Erebus', ' The Moirai']
Parents:
['Hydros', 'Gaia']

/wiki/Chaos_(mythology)

/wiki/Chronos
Offspring:
['Chaos', ' Aether', ' Phanes', ' Erebus', ' The Moirai', ' Hemera', ' Horae']

/wiki/Erebus

/wiki/Eros
Children:
['Hedone / Voluptas']
Parents:
['Aphrodite', 'Ares']

/wiki/Hemera
Children:
['Gaia', 'Uranus', 'Thalassa', 'Pontus']
Parents:
['Erebus', 'Nyx']

/wiki/Hypnos
Children:
['Morpheus', ' Phobetor', 'Phantasos']
Parents:
['Nyx', 'Erebus']

/wiki/Gaia_(mythology)
Offspring:
['Uranus', ' Pontus', ' the Ourea', ' Hecatonchires', ' Cyclopes', ' Titans', ' The Gigantes', ' Nereus', ' Thaumus', ' Phorcys', ' Ceto', ' Eurybia', ' Aergia', ' Typhon', 'Python']
Parents:
['None', 'Chaos ', 'Aether', 'Hemera

In [34]:
 from py2neo import Graph


In [35]:
v = "/wiki/Achlys"
v[v.rfind("/") + 1:]

'Achlys'

In [36]:
def fill_in_missing(uri):
    print(uri)
    f = open(uri + '.html','r')
    sub_doc = f.read()

    soup_t_sub = BeautifulSoup(str(sub_doc), 'html.parser')
    b = soup_t_sub.findAll("p")
    for i in b:
        st = re.sub(r'\<[^>]*\>' ,'',str(i))
        k = re.search(r"(\b[A-Z]+\w*\b)(?:\b[a-z]*\b\s)*(daughter|father|mother|son)(?:\s?of\s?)(?:\b[a-z]*\b\s)*(\b[A-Z]+\w+\b)(?:\sand\s)?(\b[A-Z]+\w*\b)?", st)
        #(daughter|father|mother|son)(?:\s?of\s?)(?:\b[a-z]*\b\s)*(\b[A-Z]+\w+\b)(?:\sand\s)?(\b[A-Z]+\w*\b\s)?
        if k:
            print(k.group())
        #if "son of" in str(i) or "daughter of" in str(i) or "father of" in str(i) or "mother of" in str(i):
            #print(re.sub(r'[<.*>]' ,'',str(i)))  Doesnt work
            #print("---")
            #print(st)
            
            
    print()

In [37]:
for p in prime_uris:
    fill_in_missing(p)

wiki/Achlys
If she was a daughter of Nyx

wiki/Aion_(deity)
Aion the son of Zeus

wiki/Aether_(mythology)
Aether was the son of Chaos and Caligo
Aether was the son of Erebus

wiki/Ananke_(mythology)
Considered as the mother of the Fates

wiki/Chaos_(mythology)
Chaos as the son of Chronus and Ananke

wiki/Chronos

wiki/Erebus
Erebus as the father of Geras

wiki/Eros
Eros is represented as the son of Aphrodite

wiki/Hemera

wiki/Hypnos
Hypnos is the son of Nyx

wiki/Gaia_(mythology)

wiki/Nemesis_(mythology)
There she was a daughter of Oceanus
Nemesis has been described as the daughter of Oceanus
Nemesis being the mother of Helen
Zeus the son of Kronos
Nemesis was the mother of Helene
Nemesis was the mother of the Telchines

wiki/Nesoi_(mythology)

wiki/Nyx_(mythology)
Here she is also the mother of Eros

wiki/Uranus_(mythology)
Uranus was the son of Nyx
Aphrodite was the daughter of Zeus and Dione
Olympus and she found the son of Kronos

wiki/Ourea

wiki/Phanes_(mythology)

wiki/Pontus_